<a href="https://colab.research.google.com/github/0somens/Analysis/blob/main/Correcci%C3%B3n_Dataset_Ropa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os, shutil, random
from sklearn.model_selection import train_test_split

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
clothes_root = "/content/drive/MyDrive/DatasetImg/Train/Ropa"  # carpeta que contiene subfolders Polo, Rok, Sweater...
combined_root = "/content/drive/MyDrive/DatasetImg/dataset_combined"  # salida


In [14]:
# Parámetros de split
train_frac = 0.7
val_frac = 0.15
test_frac = 0.15
random_state = 42

In [15]:
for split in ['train','validation','test']:
    os.makedirs(os.path.join(combined_root, split, 'prendas'), exist_ok=True)


In [16]:
all_files = []
for sub in os.listdir(clothes_root):
    subp = os.path.join(clothes_root, sub)
    if os.path.isdir(subp):
        files = [os.path.join(subp, f) for f in os.listdir(subp)
                 if f.lower().endswith(('.jpg','.jpeg','.png'))]
        all_files.extend(files)

In [17]:
print("Total imágenes ropa:", len(all_files))

Total imágenes ropa: 7500


In [18]:
# Shuffle y split
train_files, temp = train_test_split(all_files, train_size=train_frac, random_state=random_state)
val_size = val_frac / (val_frac + test_frac)
val_files, test_files = train_test_split(temp, train_size=val_size, random_state=random_state)


In [19]:
# Copiar a carpetas destino
def copy_list(file_list, dst_dir):
    for src in file_list:
        fn = os.path.basename(src)
        dst = os.path.join(dst_dir, fn)
        # Si ya existe un nombre igual, renombrar para evitar sobreescritura
        if os.path.exists(dst):
            base, ext = os.path.splitext(fn)
            i = 1
            while os.path.exists(os.path.join(dst_dir, f"{base}_{i}{ext}")):
                i += 1
            dst = os.path.join(dst_dir, f"{base}_{i}{ext}")
        shutil.copy(src, dst)

In [20]:
copy_list(train_files, os.path.join(combined_root, 'train', 'prendas'))
copy_list(val_files, os.path.join(combined_root, 'validation', 'prendas'))
copy_list(test_files, os.path.join(combined_root, 'test', 'prendas'))

In [21]:
print("Copiado: train", len(train_files), "val", len(val_files), "test", len(test_files))

Copiado: train 5250 val 1125 test 1125
